In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords

#Importing Dependencies for Feature Engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
from prettytable import PrettyTable
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Data Load

In [ ]:
%%time
df = pd.read_csv(r'IMDB-Dataset.csv', encoding='latin-1')

# Data Preprocessing and Data Cleaning


In [ ]:
#customize stopword as per data
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
new_stopwords = ["la la la","sunday night", "friday night", "bla bla bla", "movie","one","film","would","shall","could","might"]
stop_words.extend(new_stopwords)
stop_words.remove("not")
stop_words=set(stop_words)

In [ ]:
""'Data Cleaning and Preprocessing pipeline'""

#Remove Special Characters
def remove_special_character(content):
    return re.sub('\[[^@#$&!%*]*\]', '', content)

#Removing URL's
def remove_url(content):
    return re.sub(r'http\S+', '', content)

#Removing the Stopwords from Content
def remove_stopwords(content):
    clean_data= []
    for i in content.split():
        if i.strip().lower() not in stop_words and i.strip().lower().isalpha():
            clean_data.append(i.strip().lower())
    return " ".join(clean_data)

# Expension of English Characters
def contraction_expansion(content):
    content = re.sub(r"won\'t", "would not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"don\'t", "do not", content)
    content = re.sub(r"shouldn\'t", "should not", content)
    content = re.sub(r"needn\'t", "need not", content)
    content = re.sub(r"hasn\'t", "has not", content)
    content = re.sub(r"haven\'t", "have not", content)
    content = re.sub(r"werent\'t", "were not", content)
    content = re.sub(r"mightn\'t", "might not", content)
    content = re.sub(r"didn\'t", "did not", content)
    content = re.sub(r"n\'t", " not", content)
    content = re.sub(r"\'re", "are", content)
    content = re.sub(r"\'s", "is", content)
    content = re.sub(r"\'d", "would", content)
    content = re.sub(r"\'ll", "will", content)
    content = re.sub(r"\'ve", "have", content)
    content = re.sub(r"\'m", "am", content)
    return content

# Data Preprocessing
def data_cleaning(content):
    content = remove_special_character(content)
    content = remove_url(content)
    content = contraction_expansion(content)            #Contraction_expansion() must be run first then run remove_stopwords() 
    content = remove_stopwords(content)
    return content
    

In [ ]:
%%time
# Data Cleaning
df['Reviews_clean'] = df['Reviews'].apply(data_cleaning)
df['Reviews_clean'].head(5)

# Data OverView

In [ ]:
# Checkin Missing Values
df.isna().sum()

In [ ]:
df['Ratings'].describe()

In [ ]:
df['Reviews_clean'].describe()

In [ ]:
print('Unique Reviews:%s' % df.Reviews_clean.nunique())
print('Unique Movies:%s' % df.Movies.nunique())
print('No. of Ratings:%s' % df.Ratings.count())

# Expolatory Data Analysis

In [ ]:
# Checking Class Imbalancing
sns.countplot(x=df["Ratings"])
plt.show()
print(df["Ratings"].value_counts())

In [ ]:
%%time
"""Visualization of Important Words From Positive Review"""
import seaborn as sns
from collections import Counter

words = ' '.join(map(str, df["Reviews_clean"][df["Ratings"] >= 7])).split()

# Create a counter for word frequencies
word_counter = Counter(words)

# Create a DataFrame from the counter
word_df = pd.DataFrame(list(word_counter.items()), columns=['Word', 'Frequency'])

# Sort the DataFrame by frequency in descending order
word_df = word_df.sort_values(by='Frequency', ascending=False)

# Select the top N words to visualize
top_words = 30
word_df_top = word_df.head(top_words)

# Plot the horizontal bar chart
plt.figure(figsize=(10, 8))
sns.barplot(x='Frequency', y='Word', data=word_df_top, palette='viridis')
plt.title('Top {} Words in Positive Reviews'.format(top_words))
plt.xlabel('Frequency')
plt.ylabel('Word')
plt.show()



In [ ]:
%%time
"""Visualization of Important Words From Negative Review"""
import seaborn as sns
from collections import Counter

# Assuming 'neg' contains negative reviews
words = ' '.join(map(str, df["Reviews_clean"][df["Ratings"] <= 4])).split()

# Create a counter for word frequencies
word_counter = Counter(words)

# Create a DataFrame from the counter
word_df = pd.DataFrame(list(word_counter.items()), columns=['Word', 'Frequency'])

# Sort the DataFrame by frequency in descending order
word_df = word_df.sort_values(by='Frequency', ascending=False)

# Select the top N words to visualize
top_words = 30
word_df_top = word_df.head(top_words)

# Plot the horizontal bar chart
plt.figure(figsize=(10, 8))
sns.barplot(x='Frequency', y='Word', data=word_df_top, palette='viridis')
plt.title('Top {} Words in Negative Reviews'.format(top_words))
plt.xlabel('Frequency')
plt.ylabel('Word')
plt.show()


# Statistical Review of Data


In [ ]:
#Visualization of Number of Character in Reviews
figure,(pos_ax,neg_ax)=plt.subplots(1,2,figsize=(15,18))
len_pos_review=df[df['Ratings']>=7]['Reviews_clean'].str.len()
pos_ax.hist(len_pos_review,color='green')
pos_ax.set_title('Positive Reviews')
len_neg_review=df[df['Ratings']<=4]['Reviews_clean'].str.len()
neg_ax.hist(len_neg_review,color='red')
neg_ax.set_title('Negative Reviews')
figure.suptitle('Number of Characters in Reviews')
plt.show


In [ ]:
#Visualization of Number of Word in Reviews
figure,(pos_ax,neg_ax)=plt.subplots(1,2,figsize=(15,18))
pos_word=df[df['Ratings']>=7]['Reviews_clean'].str.split().map(lambda review: len(review))
pos_ax.hist(pos_word,color='green')
pos_ax.set_title('Number of Words in Positive Reviews')
neg_word=df[df['Ratings']<=4]['Reviews_clean'].str.split().map(lambda review: len(review))
neg_ax.hist(neg_word,color='red')
neg_ax.set_title('Number of Words in Negative Reviews')
figure.suptitle('Number of Words in Reviews')
plt.show

In [ ]:
# Average of Words in Reviews 
figure,(pos_ax,neg_ax)=plt.subplots(1,2,figsize=(15,18))
pos_word=df[df['Ratings']>=7]['Reviews_clean'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(pos_word.map(lambda x :np.mean(x)), ax=pos_ax,color='green')
pos_ax.set_title('Positive Reviews')
neg_word=df[df['Ratings']<=4]['Reviews_clean'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(neg_word.map(lambda x: np.mean(x)), ax=neg_ax,color='red')
pos_ax.set_title('Negative Reviews')
figure.suptitle('Average Word Length in Reviews')


In [ ]:
#Important Features by Usin CountVectorizor
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']>=7],20,1)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common Words in Positive Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']>=7],20,2)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common Bigram Words in Positive Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']>=7],20,3)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common Trigram Words in Positive Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
pd.options.display.max_colwidth=1000
df[["Reviews_clean","Ratings","Movies"]][(df['Ratings']>=7) & (df['Reviews_clean'].str.contains('not like| not want| not good| not perfect| not fine| not excellent| not expecting| not saying| not want| not feel| not sure| not much| not even| not know| not really| not get| not think'))]

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']>=7],20,4)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common 4-gram Words in Positive Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']>=7],20,5)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common 5-gram Words in Positive Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
pd.options.display.max_colwidth=1000
df[["Reviews_clean","Ratings","Movies"]][(df['Ratings']>=7) & (df['Reviews_clean'].str.contains('mario mario mario mario| blah blah blah blah| la la la la'))]

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']<=4],20,1)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common Unigram Words in Negative Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']<=4],30,2)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common Bigram Words in Negative Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']<=4],40,3)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common Trigram Words in Negative Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']<=4],45,4)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common 4-gram Words in Negative Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
most_common_uni = get_top_text_ngrams(df.Reviews_clean[df['Ratings']<=4],50,5)
most_common_uni = dict(most_common_uni)
temp =pd.DataFrame(columns= ["Common Words", 'Count'])
temp['Common Words'] = list(most_common_uni.keys())
temp['Count'] = list(most_common_uni.values())
fig = px.bar(temp, x='Count', y='Common Words', title='Common 5-gram Words in Negative Reviews', orientation='h', width=700, height= 700,color='Common Words')
fig.show()

In [ ]:
pd.options.display.max_colwidth=1000
df[["Reviews_clean","Ratings","Movies"]][(df['Ratings']<=4) & (df['Reviews_clean'].str.contains('like| good| great'))].head(100)

In [ ]:
pd.options.display.max_colwidth=1000
df[["Reviews_clean","Ratings","Movies"]][(df['Ratings']<=4) & (df['Reviews_clean'].str.contains('saturday night friday'))].head(100)

In [ ]:
pd.options.display.max_colwidth=1000
df[["Reviews_clean","Ratings","Movies"]][(df['Ratings']>=7) & (df['Reviews_clean'].str.contains('saturday night friday'))].head(100)

# # Feature Engineering 

In [ ]:
#Mapping the Rating data as 1 is(+ve)(if rating >=7) and 0(-ve)(if Rating<=4) else 2(neutral)(If Rating=5 or 6)
df['Label'] = df['Ratings'].apply(lambda x: '1' if x>=7 else('0' if x<=4 else '2'))
#Removing
df = df[df.Label<'2']
data = df[['Reviews_clean','Label']]
print(data['Label'].value_counts())

In [ ]:
#Importing Dependencies for Feature Engineering 
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
from prettytable import PrettyTable
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


# Lemmatization


In [ ]:
#Lemmatization of Words
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

##### Vectorization with Count Vectorizer and TFIDF Vectorizer with Unigram

In [ ]:
train,test = train_test_split(data,test_size=0.3,random_state=42,shuffle=True)
countvect = CountVectorizer(analyzer='word', tokenizer= LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer='word', tokenizer= LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.fit_transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.fit_transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

##### Feature Importance with Logistic Regression and CountVectorizer with Unigram

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(countvect.get_feature_names(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)

###### Feature Importance with Logistic Regression and TFIDF Vectorizer with Unigram

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(tfidfvect.get_feature_names(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)   

# We can see that TFIDF give more score to the important words as compared to Countvect example check word "awful"--> negative word

##### Feature Importance with TFIDF and Count Vectorizer on Bigrams With Logistic Regression 

In [ ]:
train,test = train_test_split(data,test_size=0.3,random_state=42,shuffle=True)
countvect = CountVectorizer(analyzer='word', tokenizer= LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer='word', tokenizer= LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.fit_transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.fit_transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

###### Feature Importance with TFIDF  on Bigrams with Logistic Regression


In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(tfidfvect.get_feature_names(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)   


###### Feature Importance with Count Vectorizer  on Bigrams with Logistic Regression
 

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(countvect.get_feature_names(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)

In [ ]:
pd.options.display.max_colwidth=1000
df[["Reviews_clean","Ratings","Movies"]][(df['Ratings']>=9) & (df['Reviews_clean'].str.contains('bad review'))].head(50)

###### Feature Importance with TFIDF and Count Vectorizer  on Trigrams with Logistic Regression


In [ ]:
train,test = train_test_split(data,test_size=0.3,random_state=42,shuffle=True)
countvect = CountVectorizer(analyzer='word', tokenizer= LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer='word', tokenizer= LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.fit_transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.fit_transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

###### Feature Importance with Count Vectorizer  on Trigrams with Logistic Regression
 

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(countvect.get_feature_names(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)

###### Feature Importance with TFIDF Vectorizer  on Trigrams with Logistic Regression
 

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(tfidfvect.get_feature_names(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)   


###### Feature Importance with TFIDF and Count Vectorizer  on 4-grams with Logistic Regression


In [ ]:
train, test = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)

countvect = CountVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(4, 4), min_df=10, max_features=500)
tfidfvect = TfidfVectorizer(analyzer='word', tokenizer=LemmaTokenizer(), ngram_range=(4, 4), min_df=10, max_features=500)

x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()  # Use transform instead of fit_transform

x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()  # Use transform instead of fit_transform

y_train = train['Label']
y_test = test['Label']

###### Feature Importance with Count Vectorizer  on 4-grams with Logistic Regression
 

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(countvect.get_feature_names(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)

###### Feature Importance with TFIDF Vectorizer  on 4-grams with Logistic Regression
 

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(['Feature','Score'])
for feature, importantance in zip(tfidfvect.get_feature_names(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importantance])
        i=i+1
print(importantfeature)   


# Vectorization with TFIDF and Count Vectorizer with Unigram, Bigram and Trigram

In [ ]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=5000)
x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()
y_train = train['Label']
y_test = test['Label']

### Feature Selection with Chi Squared

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np
N = 5000
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1

# Model Selection 




In [ ]:
#Import Prereq Libraries for Model Selection
import sys
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

##### Logistic Regression Model 

In [ ]:
model_1 = Pipeline(
    steps=[
        ('classifier', LogisticRegression())
    ])

##### Training of Logistic Regression Model 

In [ ]:
%%time
model_1.fit(x_train_tfidf,y_train)

In [ ]:
%%time
print("Precision Score on Training dataset for Logistic Regression: %s" % precision_score(y_train, model_1.predict(x_train_tfidf),average='weighted' ))
print("AUC Score on Training dataset for Logistic Regression: %s" % roc_auc_score(y_train, model_1.predict(x_train_tfidf),average='weighted' ))
print("F1 Score on Training dataset for Logistic Regression: %s" % f1_score(y_train, model_1.predict(x_train_tfidf),average='weighted' ))

print("Precision Score on Test dataset for Logistic Regression: %s" % precision_score(y_test, model_1.predict(x_test_tfidf),average='weighted' ))
print("AUC Score on Test dataset for Logistic Regression: %s" % roc_auc_score(y_test, model_1.predict(x_test_tfidf),average='weighted' ))
print("F1 Score on Test dataset for Logistic Regression: %s" % f1_score(y_test, model_1.predict(x_test_tfidf),average='weighted' ))

##### Decision Tree Classifier

In [ ]:
model_2 = Pipeline(
    steps=[
        ('classifier', DecisionTreeClassifier())
    ])

##### Training on Decision Tree  

In [ ]:
%%time
model_2.fit(x_train_tfidf,y_train)

In [ ]:
%%time
print("Precision Score on Training dataset for DecisionTreeClassifier: %s" % precision_score(y_train, model_2.predict(x_train_tfidf),average='weighted' ))
print("AUC Score on Training dataset for DecisionTreeClassifier: %s" % roc_auc_score(y_train, model_2.predict(x_train_tfidf),average='weighted' ))
print("F1 Score on Training dataset for DecisionTreeClassifier: %s" % f1_score(y_train, model_2.predict(x_train_tfidf),average='weighted' ))

print("Precision Score on Test dataset for DecisionTreeClassifier: %s" % precision_score(y_test, model_2.predict(x_test_tfidf),average='weighted' ))
print("AUC Score on Test dataset for DecisionTreeClassifier: %s" % roc_auc_score(y_test, model_2.predict(x_test_tfidf),average='weighted' ))
print("F1 Score on Test dataset for Logistic DecisionTreeClassifier: %s" % f1_score(y_test, model_2.predict(x_test_tfidf),average='weighted' ))

##### Random Forest Classifier 

In [ ]:
model_3 = Pipeline(
    steps=[
        ('classifier', RandomForestClassifier())
    ])

##### Training on Random Forest 

In [ ]:
%%time
model_3.fit(x_train_tfidf,y_train)

In [ ]:
%%time
print("Precision Score on Training dataset for RandomForestClassifier: %s" % precision_score(y_train, model_3.predict(x_train_tfidf),average='weighted' ))
print("AUC Score on Training dataset for RandomForestClassifier: %s" % roc_auc_score(y_train, model_3.predict(x_train_tfidf),average='weighted' ))
print("F1 Score on Training dataset for RandomForestClassifier: %s" % f1_score(y_train, model_3.predict(x_train_tfidf),average='weighted' ))

print("Precision Score on Test dataset for RandomForestClassifier: %s" % precision_score(y_test, model_3.predict(x_test_tfidf),average='weighted' ))
print("AUC Score on Test dataset for RandomForestClassifier: %s" % roc_auc_score(y_test, model_3.predict(x_test_tfidf),average='weighted' ))
print("F1 Score on Test dataset for Logistic RandomForestClassifier: %s" % f1_score(y_test, model_3.predict(x_test_tfidf),average='weighted' ))

##### AdaBoost Classifier 

In [ ]:
model_4 = Pipeline(
    steps=[
        ('classifier', AdaBoostClassifier())
    ])

##### Training on Adaboost Classifier 

In [ ]:
%%time
model_4.fit(x_train_tfidf,y_train)

In [ ]:
%%time
print("Precision Score on Training dataset for AdaBoostClassifier: %s" % precision_score(y_train, model_4.predict(x_train_tfidf),average='weighted' ))
print("AUC Score on Training dataset for AdaBoostClassifier: %s" % roc_auc_score(y_train, model_4.predict(x_train_tfidf),average='weighted' ))
print("F1 Score on Training dataset for AdaBoostClassifier: %s" % f1_score(y_train, model_4.predict(x_train_tfidf),average='weighted' ))

print("Precision Score on Test dataset for AdaBoostClassifier: %s" % precision_score(y_test, model_3.predict(x_test_tfidf),average='weighted' ))
print("AUC Score on Test dataset for AdaBoostClassifier: %s" % roc_auc_score(y_test, model_4.predict(x_test_tfidf),average='weighted' ))
print("F1 Score on Test dataset for Logistic AdaBoostClassifier: %s" % f1_score(y_test, model_4.predict(x_test_tfidf),average='weighted' ))

##### Decision Tree with max depth 11 to overcome overfit 

In [ ]:
model_3 = Pipeline(
    steps=[
        ('classifier', DecisionTreeClassifier(criterion='gini', max_depth=11, min_samples_split=2, min_samples_leaf=1))
    ])

In [ ]:
%%time
model_3.fit(x_train_tfidf,y_train)

In [ ]:
%%time
print("Precision Score on Training dataset for DecisionTreeClassifier: %s" % precision_score(y_train, model_2.predict(x_train_tfidf),average='weighted' ))
print("AUC Score on Training dataset for DecisionTreeClassifier: %s" % roc_auc_score(y_train, model_2.predict(x_train_tfidf),average='weighted' ))
print("F1 Score on Training dataset for DecisionTreeClassifier: %s" % f1_score(y_train, model_2.predict(x_train_tfidf),average='weighted' ))

print("Precision Score on Test dataset for DecisionTreeClassifier: %s" % precision_score(y_test, model_2.predict(x_test_tfidf),average='weighted' ))
print("AUC Score on Test dataset for DecisionTreeClassifier: %s" % roc_auc_score(y_test, model_2.predict(x_test_tfidf),average='weighted' ))
print("F1 Score on Test dataset for Logistic DecisionTreeClassifier: %s" % f1_score(y_test, model_2.predict(x_test_tfidf),average='weighted' ))

# Hyper Parameter Tunning with Grid Search

In [ ]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection

def hyperparamtune(classifier, param_grid, metric, verbose_value,cv):
    model = model_selection.GridSearchCV(estimator=classifier,
                                        param_grid=param_grid,
                                        scoring=metric,
                                        verbose=verbose_value,
                                        cv=cv)
    
    model.fit(x_train_tfidf,y_train)
    print("Best Score %s" % {model.best_score_})
    print("Best Hyperparameter set: ")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")
    return model, best_parameters   

##### HyperParameter Tunning of Logistic Regression

In [ ]:
%%time
param_gd = {"penalty":["12","11"],
           "C":[0.01,0.1,1.0,10],
           "tol":[0.0001,0.001,0.01],
           "max_iter":[100,200]}
model_7, best_param = hyperparamtune(LogisticRegression(),param_gd,"accuracy",10,5)

##### Evaluation of FineTuned Logistic Regression Model

In [ ]:
%%time
print("Precision Score on Training dataset for Logistic Regression: %s" % precision_score(y_train, model_7.predict(x_train_tfidf),average='weighted' ))
print("AUC Score on Training dataset for Logistic Regression: %s" % roc_auc_score(y_train, model_7.predict(x_train_tfidf),average='weighted' ))
print("F1 Score on Training dataset for Logistic Regression: %s" % f1_score(y_train, model_7.predict(x_train_tfidf),average='weighted' ))

print("Precision Score on Test dataset for Logistic Regression: %s" % precision_score(y_test, model_7.predict(x_test_tfidf),average='weighted' ))
print("AUC Score on Test dataset for Logistic Regression: %s" % roc_auc_score(y_test, model_7.predict(x_test_tfidf),average='weighted' ))
print("F1 Score on Test dataset for Logistic Regression: %s" % f1_score(y_test, model_7.predict(x_test_tfidf),average='weighted' ))

In [ ]:
# If System Memory giving space easily then also do this fine tuning on AdaBoost And Decision Tree and other classifier 
# Fine Tuned Logistic Regression Classifier Giving Best F1 Score.....

In [ ]:
train,test=train_test_split(data,test_size=.3,random_state=42, shuffle=True)
#countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=10000)
#x_train_count = countvect.fit_transform(train['Reviews_clean']).toarray()
#x_test_count = countvect.transform(test['Reviews_clean']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews_clean']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews_clean']).toarray()

y_train = train['Label']
y_test = test['Label']

# Model Evaluation Model_1 is without Pipeline and Model_2 is with Pipeline on Logistic Regression

In [ ]:
model_1=LogisticRegression(penalty='l2',dual=False, tol=0.0001, C=10, solver='lbfgs', max_iter=200, multi_class='auto', verbose=0, warm_start=False, n_jobs=None)
model_2=Pipeline(
    steps=[
        #best base model("classifier", LogisticRegression(penalty='l2',dual=False, tol=0.0001, C=1.0, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None)),
    ('vect',TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=10000)),("classifier", LogisticRegression(penalty='l2',dual=False, tol=0.0001, C=10, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None))]
)

In [ ]:
%%time
model_1.fit(x_train_tfidf,y_train)

In [ ]:
%%time
model_2.fit(train['Reviews_clean'],y_train)

In [ ]:
%%time
print("Precision Score for Logistic Regression: %s" % precision_score(y_test,model_1.predict(x_test_tfidf),average='micro'))
print("Recall Score for Logistic Regression: %s" % recall_score(y_test,model_1.predict(x_test_tfidf),average='micro'))
print("AUC Score for Logistic Regression: %s" % roc_auc_score(y_test,model_1.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_1 =f1_score(y_test,model_1.predict(x_test_tfidf),average="weighted")
print("F1 Score for Logistic Regression: %s" % f1_score_1)
print("Accuracy Score for Logistic Regression: %s" % accuracy_score(y_test,model_1.predict(x_test_tfidf)))
print("Precision Score for Logistic Regression Pipeline: %s" % precision_score(y_test,model_2.predict(test['Reviews_clean']),average='micro'))
print("Recall Score for Logistic Regression Pipeline: %s" % recall_score(y_test,model_2.predict(test['Reviews_clean']),average='micro'))
print("AUC Score for Logistic Regression Pipeline: %s" % roc_auc_score(y_test,model_2.predict_proba(test['Reviews_clean'])[:,1],multi_class='ovo',average='macro'))
f1_score_2 =f1_score(y_test,model_2.predict(test['Reviews_clean']),average="weighted")
print("F1 Score for Logistic Regression Pipeline: %s" % f1_score_2)
print("Accuracy Score for Logistic Regression Pipeline: %s" % accuracy_score(y_test,model_2.predict(test['Reviews_clean'])))

In [ ]:
y_predict=model_1.predict(x_test_tfidf)
y_predict_prob=model_1.predict_proba(x_test_tfidf)[:,1]

# Confusion Matrics

In [ ]:
def confusion_matrix_plot(y_test,y_score):
    confmatrix = confusion_matrix(y_test,y_score)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(confmatrix)
    ax.grid(False)
    ax.xaxis.set(ticks=(0, 1), ticklabels=('Predicted 0s', 'Predicted 1s'))
    ax.yaxis.set(ticks=(0, 1), ticklabels=('Actual 0s', 'Actual 1s'))
    ax.set_ylim(1.5, -0.5)
    for i in range(2):
        for j in range(2):
            ax.text(j, i, confmatrix[i, j], ha='center', va='center', color='red')
    plt.show()

In [ ]:
confusion_matrix_plot(y_test,y_predict)